In [1]:
### Load packages and preprocess data using professor's tsp-starter

## Load packages
using HiGHS, JuMP, Graphs

## Read data
ftv33 = "ftv33.atsp"
kro124p = "kro124p.atsp"
si175 = "si175.atsp"

## Preprocess data
strdata_ftv = split(read(ftv33, String))
strdata_kro = split(read(kro124p, String))
strdata_si = split(read(si175, String))

intdata_ftv = [parse(Int,s) for s in strdata_ftv]
intdata_kro = [parse(Int,s) for s in strdata_kro]
intdata_si = [parse(Int,s) for s in strdata_si]

# Number of nodes
n_ftv = intdata_ftv[1]
n_kro = intdata_kro[1]
n_si = intdata_si[1]

# Define nodeset
nodes_ftv = 1:n_ftv
nodes_kro = 1:n_kro
nodes_si = 1:n_si

# Convert data structure as Set and define cost matrix
nodeset_ftv = Set(nodes_ftv) 
c_ftv = zeros(Int64,n_ftv,n_ftv)
[c_ftv[i,j] = intdata_ftv[1+n_ftv*(i-1)+j] for i in nodes_ftv for j in nodes_ftv]

nodeset_kro = Set(nodes_kro) 
c_kro = zeros(Int64,n_kro,n_kro)
[c_kro[i,j] = intdata_kro[1+n_kro*(i-1)+j] for i in nodes_kro for j in nodes_kro]

nodeset_si = Set(nodes_si) 
c_si = zeros(Int64,n_si,n_si)
[c_si[i,j] = intdata_si[1+n_si*(i-1)+j] for i in nodes_si for j in nodes_si]

30625-element Vector{Int64}:
 9999999
     113
     189
     299
     189
     177
     187
     187
     187
     162
     187
     162
     187
       ⋮
     143
     181
     198
     213
     309
     279
     289
     299
     319
     328
     337
 9999999

In [2]:
### Compact formulation_ftv33

## Create the model
m_comp_ftv = Model()

## Declare all the decision variables
@variable(m_comp_ftv, x[nodes_ftv,nodes_ftv], Bin)

# Position variables to eliminate the subtours
@variable(m_comp_ftv, S[nodes_ftv], Int)

## Define the objective
@objective(m_comp_ftv, Min, sum(c_ftv[i,j]x[i,j] for i in nodes_ftv, j in nodes_ftv))

## Write all the constraints
# Flow in & out
@constraint(m_comp_ftv, flow_out[i in nodes_ftv],
        sum(x[i,j] for j in nodes_ftv) == 1)

@constraint(m_comp_ftv, flow_in[i in nodes_ftv],
        sum(x[j,i] for j in nodes_ftv) == 1)

# Eliminate the subtour
@constraint(m_comp_ftv, initial_val, S[1] == 0)

@constraint(m_comp_ftv, subtour_el[i in nodes_ftv, j in 2:n_ftv],
        S[i] + 1 <= S[j] + n_ftv*(1-x[i,j]))

@constraint(m_comp_ftv, seq_cond[i in nodes_ftv],
        0 <= S[i] <= n_ftv - 1)

## Solve model
set_optimizer(m_comp_ftv, HiGHS.Optimizer)
set_time_limit_sec(m_comp_ftv, 300)

optimize!(m_comp_ftv)

## Print required results
println("---------------------------------------")

# Print Optimal value
println("Optimal value: ", objective_value(m_comp_ftv))

# Print Solution time
println("Solution time: ", solve_time(m_comp_ftv))

# Print Number of Branch-and-bound nodes
println("Number of Branch-and-bound nodes: ", node_count(m_comp_ftv))

# Selected arcs
x_comp_ftv = value.(x)
for i in nodes_ftv, j in nodes_ftv 
    if x_comp_ftv[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1157 rows, 1156 cols, 5480 nonzeros
1157 rows, 1156 cols, 5480 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   1157 rows
   1156 cols (1123 binary, 33 integer, 0 implied int., 0 continuous)
   5480 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
         0       0         0   0.00%   1187.647059     inf                  inf        0      0      2       107     0.0s
 L       0       0         0   0.00%   1272.162727     1286               1.08%     1285     83    436       554     2.9s

85.7% inactive integer columns, restarting
Model after resta

In [3]:
### Compact formulation_kro124p

## Create the model
m_comp_kro = Model()

## Declare all the decision variables
@variable(m_comp_kro, x[nodes_kro,nodes_kro], Bin)

# Position variables to eliminate the subtours
@variable(m_comp_kro, S[nodes_kro], Int)

## Define the objective
@objective(m_comp_kro, Min, sum(c_kro[i,j]x[i,j] for i in nodes_kro, j in nodes_kro))

## Write all the constraints
# Flow in & out
@constraint(m_comp_kro, flow_out[i in nodes_kro],
        sum(x[i,j] for j in nodes_kro) == 1)

@constraint(m_comp_kro, flow_in[i in nodes_kro],
        sum(x[j,i] for j in nodes_kro) == 1)

# Eliminate the subtour
@constraint(m_comp_kro, initial_val, S[1] == 0)

@constraint(m_comp_kro, subtour_el[i in nodes_kro, j in 2:n_kro],
        S[i] + 1 <= S[j] + n_kro*(1-x[i,j]))

@constraint(m_comp_kro, seq_cond[i in nodes_kro],
        0 <= S[i] <= n_kro - 1)

## Solve model
set_optimizer(m_comp_kro, HiGHS.Optimizer)
set_time_limit_sec(m_comp_kro, 300)

optimize!(m_comp_kro)

## Print required results
println("---------------------------------------")

# Print Optimal value
println("Optimal value: ", objective_value(m_comp_kro))

# Print Solution time
println("Solution time: ", solve_time(m_comp_kro))

# Print Number of Branch-and-bound nodes
println("Number of Branch-and-bound nodes: ", node_count(m_comp_kro))

# Selected arcs
x_comp_kro = value.(x)
for i in nodes_kro, j in nodes_kro 
    if x_comp_kro[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10001 rows, 10000 cols, 49106 nonzeros
10001 rows, 10000 cols, 49106 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   10001 rows
   10000 cols (9901 binary, 99 integer, 0 implied int., 0 continuous)
   49106 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.3s
         0       0         0   0.00%   34008.5875      inf                  inf        0      0      2       352     0.5s
         0       0         0   0.00%   34914.9         inf                  inf     1131    107    518       574     5.9s
         0       0         0   0.00%   35685.686353 

In [4]:
### Compact formulation_si175

## Create the model
m_comp_si = Model()

## Declare all the decision variables
@variable(m_comp_si, x[nodes_si,nodes_si], Bin)

# Position variables to eliminate the subtours
@variable(m_comp_si, S[nodes_si], Int)

## Define the objective
@objective(m_comp_si, Min, sum(c_si[i,j]x[i,j] for i in nodes_si, j in nodes_si))

## Write all the constraints
# Flow in & out
@constraint(m_comp_si, flow_out[i in nodes_si],
        sum(x[i,j] for j in nodes_si) == 1)

@constraint(m_comp_si, flow_in[i in nodes_si],
        sum(x[j,i] for j in nodes_si) == 1)

# Eliminate the subtour
@constraint(m_comp_si, initial_val, S[1] == 0)

@constraint(m_comp_si, subtour_el[i in nodes_si, j in 2:n_si],
        S[i] + 1 <= S[j] + n_si*(1-x[i,j]))

@constraint(m_comp_si, seq_cond[i in nodes_si],
        0 <= S[i] <= n_si - 1)

## Solve model
set_optimizer(m_comp_si, HiGHS.Optimizer)
set_time_limit_sec(m_comp_si, 300)

optimize!(m_comp_si)

## Print required results
println("---------------------------------------")

# Print Optimal value
println("Optimal value: ", objective_value(m_comp_si))

# Print Solution time
println("Solution time: ", solve_time(m_comp_si))

# Print Number of Branch-and-bound nodes
println("Number of Branch-and-bound nodes: ", node_count(m_comp_si))

# Selected arcs
x_comp_si = value.(x)
for i in nodes_si, j in nodes_si 
    if x_comp_si[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
30626 rows, 30625 cols, 151556 nonzeros
30626 rows, 30625 cols, 151556 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   30626 rows
   30625 cols (30451 binary, 174 integer, 0 implied int., 0 continuous)
   151556 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.8s
         0       0         0   0.00%   20251.434286    inf                  inf        0      0      2       533     1.2s
         0       0         0   0.00%   20562.705714    inf                  inf      872    286    212      7677    34.5s
         0       0         0   0.00%   20562.70

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [10]:
### Functions for delayed constraint generation using professor's tsp-starter and robust-knap

## Finding epsilon value for the heuristic
function finduniquevals(xvals)
    flatvals=reduce(vcat,xvals)
    svals=sort(flatvals[:])
    newvals = []
    curval = 0.0
    for i in 1:length(svals)
        if svals[i] > curval + 0.00001
            curval=svals[i]
            push!(newvals,curval)
        end
    end
    return newvals   
end

## Searching for violated cuts
function findviolcuts(m,xvals,nodes)

    # Define number of cuts value 
    num_cut = 0
    
    # Define epsilon using 'finduniqevals' function
    for epsilon in finduniquevals(xvals)
        # Find connected subset and its complementary set
        println("epsilon is ", epsilon)
        arcs = [(i,j) for i in nodes for j in nodes if xvals[i,j] >= epsilon - 0.00001]
        g = SimpleDiGraph(Edge.(arcs))
        comp = Set(weakly_connected_components(g))
        nodeset = Set(nodes)

        # Convert data structure as vector
        vec_comp = []
        vec_diff = []
        for s in 1:length(comp)
            push!(vec_comp, collect(comp)[s])
            push!(vec_diff, collect(setdiff(nodeset,collect(comp)[s])))
        end
    
        # Cardinality condition
        for s in 1:length(vec_comp)
            if 2 <= length(vec_comp[s]) <= length(nodeset) - 1
                # Define lhs
                lhs = sum(xvals[i,j] for i in vec_comp[s] for j in vec_diff[s])
                # If violated
                if lhs < 1 - 0.00001
                    # Add constraint
                    @constraint(m, sum(x[i,j] for i in vec_comp[s] for j in vec_diff[s]) >= 1)
                    # Increase cut number
                    num_cut = num_cut + 1
                end
            end
        end
    end
    return num_cut
end

## Loop for solving the relaxed model, searching for cuts, and adding them when found
function run_cut_gen_loop(m,nodes)
    println("running cut gen loop")
    cut_result=true
    iters=1
    total_cut = 0
    while(cut_result>0)     
        optimize!(m)
        if termination_status(m) != OPTIMAL
            cut_result=false
            println("error: model failed to solve")
        end
        println("iter", iters, ": relaxation optimal value: ", objective_value(m))
        xvals=value.(x)
        cut_result = findviolcuts(m,xvals,nodes)
        # Update total number of cut
        total_cut = total_cut + cut_result
        iters=iters+1
    end
    # Print total number of cuts added in the phase
    println("Total number of cuts is ", total_cut)
end

run_cut_gen_loop (generic function with 1 method)

In [13]:
### Delayed constraint generation_ftv33

## Declare start time
start=time()

## Create the base model
m_dcg_ftv = Model()

# Set solver and time limit 
set_optimizer(m_dcg_ftv, HiGHS.Optimizer)
set_time_limit_sec(m_dcg_ftv, 300)

## Declare the decision variables
@variable(m_dcg_ftv, 0 <= x[nodes_ftv,nodes_ftv] <= 1, Bin)

## Define the objective
@objective(m_dcg_ftv, Min, sum(c_ftv[i,j]x[i,j] for i in nodes_ftv, j in nodes_ftv))

## Write the constraint of relaxed model
# Arcs i,i should fix to 0)
@constraint(m_dcg_ftv, ident[i in nodes_ftv],
    x[i,i] == 0)

# Flow in & out
@constraint(m_dcg_ftv, flow_out[i in nodes_ftv],
    sum(x[i,j] for j in nodes_ftv) == 1)

@constraint(m_dcg_ftv, flow_in[i in nodes_ftv],
    sum(x[j,i] for j in nodes_ftv) == 1)

## Solve model 
set_silent(m_dcg_ftv)

# Phase 1
undo = relax_integrality(m_dcg_ftv);
println("---------------------------------------")
println("Phase 1: LP-Relaxation")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_ftv, nodes_ftv)

# Phase 2
undo()
println("---------------------------------------")
println("Phase 2: IP")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_ftv, nodes_ftv)

## Print required results
println("---------------------------------------")

# Print Solution time
println("total time = ", time()-start)

# Print Optimal value
if termination_status(m_dcg_ftv) == OPTIMAL
    println("optimal value: ", objective_value(m_dcg_ftv))
elseif has_values(m_dcg_ftv)
    println("not solved to optimality, but a feasible solution is found")
    println("feasible solution value: ", objective_value(m_dcg_ftv))
    println("bound on best possible value: ", objective_bound(m_dcg_ftv))
else
    println("no solution found")
end

# Selected arcs
x_dcg_ftv = value.(x)
for i in nodes_ftv, j in nodes_ftv 
    if x_dcg_ftv[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

---------------------------------------
Phase 1: LP-Relaxation
---------------------------------------
running cut gen loop
iter1: relaxation optimal value: 1185.0
epsilons are Any[1.0]
arcs are [(1, 2), (2, 3), (3, 4), (4, 1), (5, 6), (6, 7), (7, 5), (8, 9), (9, 11), (10, 33), (11, 10), (12, 32), (13, 14), (14, 13), (15, 16), (16, 17), (17, 15), (18, 12), (19, 18), (20, 23), (21, 22), (22, 21), (23, 27), (24, 20), (25, 24), (26, 25), (27, 28), (28, 29), (29, 30), (30, 26), (31, 34), (32, 19), (33, 8), (34, 31)]
g is SimpleDiGraph{Int64}(34, [[2], [3], [4], [1], [6], [7], [5], [9], [11], [33], [10], [32], [14], [13], [16], [17], [15], [12], [18], [23], [22], [21], [27], [20], [24], [25], [28], [29], [30], [26], [34], [19], [8], [31]], [[4], [1], [2], [3], [7], [5], [6], [33], [8], [11], [9], [18], [14], [13], [17], [15], [16], [19], [32], [24], [22], [21], [20], [25], [26], [30], [23], [27], [28], [29], [34], [12], [10], [31]])
comp is Set([[15, 16, 17], [8, 9, 10, 11, 33], [5, 6, 7], 

In [11]:
### Delayed constraint generation_kro124p

## Declare start time
start=time()

## Create the base model
m_dcg_kro = Model()

# Set solver and time limit 
set_optimizer(m_dcg_kro, HiGHS.Optimizer)
set_time_limit_sec(m_dcg_kro, 300)

## Declare the decision variables
@variable(m_dcg_kro, 0 <= x[nodes_kro,nodes_kro] <= 1, Bin)

## Define the objective
@objective(m_dcg_kro, Min, sum(c_kro[i,j]x[i,j] for i in nodes_kro, j in nodes_kro))

## Write the constraint of relaxed model
# Arcs i,i should fix to 0)
@constraint(m_dcg_kro, ident[i in nodes_kro],
    x[i,i] == 0)

# Flow in & out
@constraint(m_dcg_kro, flow_out[i in nodes_kro],
    sum(x[i,j] for j in nodes_kro) == 1)

@constraint(m_dcg_kro, flow_in[i in nodes_kro],
    sum(x[j,i] for j in nodes_kro) == 1)

## Solve model 
set_silent(m_dcg_kro)

# Phase 1
undo = relax_integrality(m_dcg_kro);
println("---------------------------------------")
println("Phase 1: LP-Relaxation")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_kro, nodes_kro)

# Phase 2
undo()
println("---------------------------------------")
println("Phase 2: IP")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_kro, nodes_kro)

## Print required results
println("---------------------------------------")

# Print Solution time
println("total time = ", time()-start)

# Print Optimal value
if termination_status(m_dcg_kro) == OPTIMAL
    println("optimal value: ", objective_value(m_dcg_kro))
elseif has_values(m_dcg_kro)
    println("not solved to optimality, but a feasible solution is found")
    println("feasible solution value: ", objective_value(m_dcg_kro))
    println("bound on best possible value: ", objective_bound(m_dcg_kro))
else
    println("no solution found")
end

# Selected arcs
x_dcg_kro = value.(x)
for i in nodes_kro, j in nodes_kro 
    if x_dcg_kro[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

---------------------------------------
Phase 1: LP-Relaxation
---------------------------------------
running cut gen loop
iter1: relaxation optimal value: 33978.0
epsilons are Any[1.0]
epsilon is 1.0
iter2: relaxation optimal value: 35393.0
epsilons are Any[1.0]
epsilon is 1.0
iter3: relaxation optimal value: 35798.416666666664
epsilons are Any[0.16666666666666652, 0.25, 0.33333333333333304, 0.4999999999999998, 0.6666666666666665, 0.75, 0.8333333333333333, 1.0]
epsilon is 0.16666666666666652
epsilon is 0.25
epsilon is 0.33333333333333304
epsilon is 0.4999999999999998
epsilon is 0.6666666666666665
epsilon is 0.75
epsilon is 0.8333333333333333
epsilon is 1.0
iter4: relaxation optimal value: 35896.416666666664
epsilons are Any[0.08333333333333304, 0.11111111111111119, 0.1388888888888889, 0.19444444444444448, 0.24999999999999978, 0.33333333333333326, 0.38888888888888884, 0.47222222222222215, 0.49999999999999956, 0.5833333333333333, 0.6111111111111109, 0.6666666666666665, 0.74999999999999

In [ ]:
### Delayed constraint generation_si175

## Declare start time
start=time()

## Create the base model
m_dcg_si = Model()

# Set solver and time limit 
set_optimizer(m_dcg_si, HiGHS.Optimizer)
set_time_limit_sec(m_dcg_si, 300)

## Declare the decision variables
@variable(m_dcg_si, 0 <= x[nodes_si,nodes_si] <= 1, Bin)

## Define the objective
@objective(m_dcg_si, Min, sum(c_si[i,j]x[i,j] for i in nodes_si, j in nodes_si))

## Write the constraint of relaxed model
# Arcs i,i should fix to 0)
@constraint(m_dcg_si, ident[i in nodes_si],
    x[i,i] == 0)

# Flow in & out
@constraint(m_dcg_si, flow_out[i in nodes_si],
    sum(x[i,j] for j in nodes_si) == 1)

@constraint(m_dcg_si, flow_in[i in nodes_si],
    sum(x[j,i] for j in nodes_si) == 1)

## Solve model 
set_silent(m_dcg_si)

# Phase 1
undo = relax_integrality(m_dcg_si);
println("---------------------------------------")
println("Phase 1: LP-Relaxation")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_si, nodes_si)

# Phase 2
undo()
println("---------------------------------------")
println("Phase 2: IP")
println("---------------------------------------")
run_cut_gen_loop(m_dcg_si, nodes_si)

## Print required results
println("---------------------------------------")

# Print Solution time
println("total time = ", time()-start)

# Print Optimal value
if termination_status(m_dcg_si) == OPTIMAL
    println("optimal value: ", objective_value(m_dcg_si))
elseif has_values(m_dcg_si)
    println("not solved to optimality, but a feasible solution is found")
    println("feasible solution value: ", objective_value(m_dcg_si))
    println("bound on best possible value: ", objective_bound(m_dcg_si))
else
    println("no solution found")
end

# Selected arcs
x_dcg_si = value.(x)
for i in nodes_si, j in nodes_si 
    if x_dcg_si[i,j] > 0.9999
    println("Arc ", i, " to ", j, " is selected.")
    end
end

---------------------------------------
Phase 1: LP-Relaxation
---------------------------------------
running cut gen loop
iter1: relaxation optimal value: 20243.0
epsilons are Any[1.0]
iter2: relaxation optimal value: 21111.0
epsilons are Any[0.5, 1.0]
iter3: relaxation optimal value: 21225.5
epsilons are Any[0.4999999999999999, 1.0]
iter4: relaxation optimal value: 21264.5
epsilons are Any[0.24999999999999867, 0.4999999999999971, 0.7499999999999987, 0.9999999999999969]
iter5: relaxation optimal value: 21282.5
epsilons are Any[0.25, 0.5, 0.75, 1.0]
iter6: relaxation optimal value: 21284.5
epsilons are Any[0.125, 0.25, 0.375, 0.49999999999999956, 0.625, 0.75, 0.875, 1.0]
iter7: relaxation optimal value: 21362.5
epsilons are Any[0.4999999999999991, 1.0]
iter8: relaxation optimal value: 21362.5
epsilons are Any[0.5, 1.0]
Total number of cuts is 140
---------------------------------------
Phase 2: IP
---------------------------------------
running cut gen loop
iter1: relaxation optimal v